# Profiling Python by Example 
## Eyal Trabelsi

# About Me 🙈


- Software Engineer at Salesforce 👷

- Big passion for python, data, and performance optimizations 🐍🤖

- Online at [medium](https://medium.com/@Eyaltra) | [twitter](https://twitter.com/eyaltra) 🌐

# Today ✨

- Profiling Introduction

- Profiling Strategies

# What's Profiling 🗣


- A profile is a set of statistics that describes how our programs is executed


- Can help optimize our code

# Profiling is not Rocket Science 🚀

# Optimization ?! Why ?🤨

- Fast is better than slow 🐇


- latency response time 200 milliseconds client roundtrip
- throughput successful traffic flow of 200 requests per seconds


- Memory efficiency is good 💾


- Saving money is awesome [💸](https://aws.amazon.com/ec2/pricing/on-demand/)


- Hardware will only take you so far 💻

# Before We Optimize ⏰

- It's actually needed 🚔


#### remember optimized code is:
- harder to write and read
- less maintainable
- buggier, more brittle

#### Optimize when
- gather requirements, there are some parts you won't be able to touch
- establish percentile SLAs: 50, 95, 99 max

- Our code is well tested 💯

- Good work takes cycles 🚲

In [1]:
%load_ext snakeviz
%load_ext memory_profiler
%load_ext line_profiler
%load_ext autoreload

%autoreload 2

amdahl law, focus on one part at a time

 # Profiling Options 📊

- **Resource:** CPU / RAM / I/O

- **Profiling Strategy:** Offline (Deterministic) 🐞/ Online (Statistical) 🌐

- **Profiling Granularity:** Program 📝 / Function 📝📝 / Line level 📝📝📝

 # Our Example 👾

- Naive Spelling Corrector

- Peter Norvig

![](https://prdeepakbabu.files.wordpress.com/2016/02/editdist.jpg?w=390&h=163)

In [2]:
import re
from collections import Counter

def words(text): 
    return re.findall(r'\w+', text.lower())

WORDS = Counter(words(open('big.txt').read()))

def P(word, N=sum(WORDS.values())): 
    return WORDS[word] / N

def candidates(word): 
    return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])

def known(words): 
    return set(w for w in words if w in WORDS)

def edits1(word):
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def edits2(word): 
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))

def word_correction(word): 
    return max(candidates(word), key=P)

def sentence_correction(sentence): 
    return " ".join(word_correction(word) for word in sentence.split(" "))


In [3]:
sentence_correction('grofilingg is not rocet Sgience')

'profiling is not rocket science'

# Casual Profiling 👕👖

- A sense of how the program run as a whole

- Allows to understand whether a problem exists


# time ⌛

- Measures the user/system time for a single run

- Built-in to python with support for ipython magic 

In [4]:
! time python script.py 'grofilingg is not rocet Sgience'


real	0m0.737s
user	0m0.672s
sys	0m0.059s


In [5]:
%time sentence_correction('grofilingg is not rocet Sgience')

CPU times: user 190 ms, sys: 1.32 ms, total: 192 ms
Wall time: 191 ms


'profiling is not rocket science'

# timeit ⌛⌛⌛

- Benchmark multiple runs of the code snippet.

- measures process CPU

- Built-in to python with support for ipython magic.

In [6]:
! python -m timeit -s "..."

50000000 loops, best of 5: 7.65 nsec per loop


In [7]:
%timeit sentence_correction('grofilingg is not rocet Sgience')

188 ms ± 1.41 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


- each run does thousand or millions of repetitions compensating for very fast operations
- it disables garbage collection for consistency


# memit ⌛⌛⌛

- Measures process Memory

- Has 3rd party ipython magic 

In [8]:
%memit sentence_correction('grofilingg is not rocet Sgience')

peak memory: 145.29 MiB, increment: 0.06 MiB


# Casual Profiling Landscape  ⛰️

| Profiler        | Metric   | Type        | Granularity | Built-in | Output       |Compatibility     | 
| :-:             | :-:      | :-:         | :-:         | :-:      |:-:           | :-:              | 
| time            | CPU      | Casual      | Program     | ✅       |   Text       |🐧 / 🍎 / Windows|               
| timeit          | CPU      | Casual      | Program     | ✅       |   Text       |🐧 / 🍎 / Windows|               
| pyperf          | CPU      | Casual      | Program     | ❌       |   Text       |🐧 / 🍎 / Windows|               
| memory_profiler | Memory   | Casual      | Program     | ❌       |   Text       |🐧 / 🍎 / Windows|               

# Casual Profiling Pros and Cons 👀


- Really easy 😃

- Allows to understand whether a problem exists 😃


- Can't pinpoint the bottleneck 😔

# Offline Profiling 🐞

- Track events like function calls, exceptions and line executions

- Deterministic

- Significant overhead

- More suitable for local debugging

# How Offline Profilers Work 🧠

- Work inside your process allow us easy access to its stack.

- Python let you specify a callback on specific interpreter events


- sys.setprofile - triggered on a function/line call (PyEval_SetProfile)

- sys.settrace - triggered on a function call(PyEval_SetTrace)

“PyEval_SetTrace is similar to PyEval_SetProfile, except the tracing function does receive line-number events.”

![](offlineprofiling)

# cProfile 🌊

- Traces every function call in a program 

- Identify time-consuming functions 

- By default measures process CPU

- Built-in to python with support for ipython magic 

- Only support single process not distributed systems or C parts

In [17]:
! python -m cProfile script.py 'grofilingg is not rocet Sgience'

In [16]:
%prun sentence_correction('grofilingg is not rocet Sgience')

# Snakeviz 🐍


- Support cProfile

- Create visualizations

- Has 3rd party ipython magic 


In [19]:
! snakeviz script.prof

In [4]:
%snakeviz sentence_correction('grofilingg is not rocet Sgience')

 
*** Profile stats marshalled to file '/var/folders/rx/kj503w_12bnft7tk1gh2cc000000gp/T/tmptsbergjj'. 
Embedding SnakeViz in this document...


# Memory profiler 💾

- For servers. high memory usage is due to leakes
    - Pattern: Slowly increasing memory usage
    - Goal: find code that doesnt free memory, fix leak
- For data processing, its the data processing
    - Pattern: Lumpy, spiky memory usage
    - Goal: find peak usage, reduce peak

- Traces every line in a specific function 


- Measures process Memory

- Identify high memory footprint lines


- Has 3rd party ipython magic 

In [13]:
! python -m memory_profiler script.py 'grofilingg is not rocet Sgience'

In [14]:
from script import sentence_correction, edits1
%mprun -f edits1 sentence_correction('grofilingg is not rocet Sgience')

# Offline Profiling Landscape  ⛰️

| Profiler        | Metric   |  Granularity | Built-in | Output       |Compatibility     | Comments      | 
| :-:             | :-:      |  :-:         | :-:      |:-:           | :-:              | :-:           
| cProfile        | CPU      |  Function    | ✅       |   Text       |🐧 / 🍎 / Windows|  Customizeable|
| yappi           | CPU      |  Function    | ❌       |   Text       |🐧 / 🍎 / Windows|Include C part |
| line_profiler   | CPU      |  Line        | ❌       |   Text       |🐧 / 🍎 / Windows|               |
| memory_profiler | Memory   |  Line        | ❌       |    Text      |🐧 / 🍎/ Windows |               | 
| filprofiler     | Memory   |  Function    | ❌       |   Flame      |   🐧 / 🍎       |               |
| snakeviz        |Visualizer|              | ❌       |Flame         |🐧 / 🍎 / Windows|               |

# Offline Profiling Pros and Cons 👀


- Can pinpoint the bottleneck 😃 

- Deterministic 😃

- High overhead 😔

- Can be noisy 😔


- Cant tell you which inputs are slow 😔


- Distorted results, only parts of ur program slowed down 😔

# Online Profiling 🌐


- Sample the program execution stack periodically

- Nondeterministic

- Requires more time to be accurate


- Marginal overhead


- More suitable for continuous production monitoring

# How Online Profilers Work 🧠


- Python let you specify a signal handler


- The setitimer system call sends a signal every X milliseconds

![](online_profiling.png)

- System calls sometimes take a few milliseconds 

- Limit your ability to sample too frequently

- Use python interpreter callbacks


- But doesn't collect stack samples every callback. 

        
    

![](https://joerick.me/img/pyinstrument-profile.png)

# pyinstrument 🎷

- Measuring process CPU

- Sample the stack every 1ms
    
    

- Doesn't have 3rd party ipython magic 

- If a function is cumulatively slow it will show up often 

- If a function is fast we wont see it at all

In [15]:
! pyinstrument script.py 'grofilingg is not rocet Sgience'


  _     ._   __/__   _ _  _  _ _/_   Recorded: 19:57:32  Samples:  241
 /_//_/// /_\ / //_// / //_'/ //     Duration: 0.724     CPU time: 1.378
/   _/                      v3.2.0

Program: script.py grofilingg is not rocet Sgience

0.723 <module>  script.py:2
├─ 0.328 words  script.py:6
│  └─ 0.324 findall  re.py:215
│     └─ 0.324 Pattern.findall  <built-in>:0
├─ 0.235 sentence_correction  script.py:41
│  └─ 0.235 <genexpr>  script.py:43
│     └─ 0.235 word_correction  script.py:37
│        └─ 0.235 candidates  script.py:15
│           └─ 0.234 known  script.py:19
│              └─ 0.234 <genexpr>  script.py:21
│                 ├─ 0.167 <genexpr>  script.py:35
│                 │  ├─ 0.108 edits1  script.py:23
│                 │  │  ├─ 0.040 <listcomp>  script.py:29
│                 │  │  ├─ 0.035 <listcomp>  script.py:30
│                 │  │  └─ 0.029 [self]  
│                 │  └─ 0.059 [self]  
│                 └─ 0.067 [self]  
├─ 0.139 __init__  collections/__init__.py:5

# Online Profiling Landscape  ⛰️

| Profiler        | Metric   | Granularity | Built-in | Output       |Compatibility    | Comments      | 
| :-:             | :-:      | :-:         | :-:      |:-:           | :-:             | :-:           |
| pyinstrument    | CPU      | Function    | ❌       |  Flame/Text  |🐧 / 🍎 / Windows|               |
|python-flamegraph| CPU      | Function    | ❌       |  Flame       |🐧 / 🍎 / Windows|               |
| pyspy           | CPU      | Function    | ❌       |  Text        |   🐧 / 🍎       |Work on running proccess               |
| vmprof          | CPU      | Line        | ❌       |    Text      |🐧 / 🍎 / Windows|               |
| austin          |CPU/Memory| Function    | ❌       |  Flame/Text  |   🐧            |Hard installation               |
| stacksampler    | Memory   | Function    | ❌       |   Flame      |🐧 / 🍎 / Windows|              



# Online Profiling Pros and Cons 👀


- Can pinpoint the bottleneck 😃

- Still introduce overhead (marginal) 😐

- Can be noisy (less than offline) 😐


- Less accurate* 😔

- Non-deterministic 😔

- Cant tell you which inputs are slow 😔


# Logging 📋

- Record whatever we want 😃

- Doesn't add a lot of overhead 😃

- Need to be add logging upfront or you are out of luck 😔


- ❗**Pro Tip**: record hot function inputs and duration

# Logging Landscape  ⛰️

| Profiler        | Built-in | Compatibility     | Comments                           | 
| :-:             | :-:      |  :-:              | :-:                                |
| logging         | ✅       | 🐧 / 🍎 / Windows |                                    |
| loguru          | ❌       | 🐧 / 🍎 / Windows |                                    |
| eliot           | ❌       | 🐧 / 🍎 / Windows | helps understand causes in the code|
| Pysnooper       | ❌       | 🐧 / 🍎 / Windows | Never use print for debugging again|

# Create your Own Profiler  🤓

- Create a custom function for cProfile

- Create a offline profiler using sys.settrace and sys.setprofile

- Create a online profiler using setitimer and ptrace

- Use pyrasite to inject python code to running process



- To run it, ptrace has to be configured as "classic ptrace permissions": 
echo 0 | sudo tee /proc/sys/kernel/yama/ptrace_scope, which is may be a security risk
- There are non-zero chances that your target Python process will crash

# We Found The Bottleneck , Now What? 🤷


- After we find the bottleneck 🕵

- "Fix the problem" 🔧

- add more hardware
- rearchitect to divide work
- adopt async
- user smarter algorithms
- write faster python
- use native python extension
- use a library with a faster implementation
- use a different python runtime

- Watch for performance regressions ↪

# Key Takeaways 🔑


- Optimize when it's actually needed 🚔

- Our code needs to be well tested  💯

- Different tools have different tradeoffs 🔨🔧

- Add logs in strategic places 🏰

- Watch for performance regressions ↪

- Good work takes cycles 🚲

# Profiling is not Rocket Science 🚀

![](https://i.pinimg.com/originals/b9/0a/79/b90a79b4c361d079144597d0bcdd61de.jpg)